In [140]:
import sys
import os
repo_path = os.path.abspath('../')
sys.path.append(repo_path)
import pandas as pd
import geopandas as gpd
import geoplot as gplt
import scripts.addSA2 as addSA2
import numpy as np

In [141]:
df = pd.read_csv(f"../data/curated/pre_processed_data.csv")
df = addSA2.addSA2(df, use_postcode = False)
df = df.loc[df["SA2"] != 0]

In [142]:
sa2 = pd.read_csv(f"../data/raw/abs_data/pop_data.csv")
sa2 = sa2[sa2["TIME_PERIOD: Time Period"] == 2021][["ASGS_2021: Region", "OBS_VALUE"]]
sa2["ASGS_2021: Region"] = sa2["ASGS_2021: Region"].apply(lambda x: x.split(':')[0])
sa2 = sa2.rename(columns = {"ASGS_2021: Region":"SA2", "OBS_VALUE":"SA2_population"})
sa2 = sa2.astype(float)
sa2

,SA2,SA2_population
10,101021010.0,5093.0
21,101041024.0,3522.0
32,102011043.0,11457.0
43,103011612.0,10803.0
54,103031075.0,0.0
...,...,...
26949,801041118.0,708.0
26960,801061066.0,0.0
26971,801071076.0,2981.0
26982,801071077.0,2691.0


In [143]:
df["count"] = 1
df = df[["suburb", "SA2", "count"]]

In [144]:
sub_count = df.groupby(["suburb","SA2"]).sum().reset_index()

In [145]:
sa2_count = df[["SA2", "count"]].groupby("SA2").sum()
sa2_count = sa2_count.rename(columns = {"count":"sa2_count"})

In [146]:
scores = sub_count.join(sa2_count, on = "SA2")
scores["SA2_fraction"] = scores["count"]/scores["sa2_count"]

In [147]:
scores = scores.join(sa2.set_index("SA2"), on = "SA2")

,suburb,SA2,count,sa2_count,SA2_fraction,SA2_population
0,abbeyard,212031558.0,1,20,0.050000,12432.0
1,abbotsford,206041506.0,4,117,0.034188,15998.0
2,abbotsford,206071139.0,46,49,0.938776,9262.0
3,abeckett street,206041504.0,18,228,0.078947,17439.0
4,abeckett street,206061136.0,1,181,0.005525,19460.0
...,...,...,...,...,...,...
2267,yarrawonga,216021414.0,11,13,0.846154,8585.0
2268,yarrowee,201011484.0,1,24,0.041667,13062.0
2269,yea,204011062.0,1,2,0.500000,4142.0
2270,yering,212011550.0,1,7,0.142857,7563.0


In [151]:
scores["suburb_population"] = scores["SA2_population"]*scores["SA2_fraction"]
suburb_pop = scores[["suburb","suburb_population"]].groupby("suburb").sum().reset_index()

In [152]:
suburb_pop

,suburb,suburb_population
0,abbeyard,621.600000
1,abbotsford,9241.878946
2,abeckett street,1484.276970
3,aberfeldie,2462.173913
4,aberfeldy,884.357143
...,...,...
978,yarrawonga,7264.230769
979,yarrowee,544.250000
980,yea,2071.000000
981,yering,1080.428571
